# Differential Expression and Pathway analysis
The goal of this notebook is to take the processed feature counts and experimental design tables and use GSEA, ROntoTools, and QuaternaryProd to generate and compare pathway analyses. This notebook runs code in R using the DESeq2 differential expression package.

1. input full featureCount tables into DESeq2
2. ROntoTools
3. GSEA
4. QuaternaryProd with STRING database

## Preparation
1. install R packages (see R install document)


## Instructions
Before running this notebook, format the feature count data with the featureCountprocessing-generic notebook. The notebooks are separated because the processing notebook runs Python code, not R. Once the processed featureCounts and experimental design table have been created:

1. Run the first cell to import the necessary libraries and define the functions used to process the data.
2. 

In [44]:
#import libraries
library("DESeq2")

#define functions for DESeq2 object creation and heatmap creation

#input and process sample data (experiment design)
input_sample_data <- function(filepath, print_debug=FALSE){
    sampleData <- read.delim(filepath)
    sampleData_v2 <- sampleData
    
    rownames(sampleData) <- sampleData$SampleID.Lab
    sampleData$Cell.Line <- factor(sampleData$Cell.Line)
    sampleData$Time.Point <- factor(sampleData$Time.Point)
    if(print_debug == TRUE){
        print(head(sampleData))
        }
    return(sampleData)
}

#Input and process raw gene counts (Feature Counts)
input_raw_counts <- function(filepath, sampleData, print_debug=FALSE){
    rawCounts <- read.delim(filepath)
    #print(head(rawCounts))
    
    geneID <- rawCounts$gene_name
    sampleIndex <- grepl("ES\\d+", colnames(rawCounts))
    rawCounts <- as.matrix(rawCounts[,sampleIndex])
    rownames(rawCounts) <- geneID
    if(print_debug == TRUE){
        print(head(rawCounts))
    }
    
    rawCounts <- rawCounts[,unique(rownames(sampleData))]
    cat("Test that dimensions of rawCounts and sampleData match:", all(colnames(rawCounts) == rownames(sampleData)), "\n")
    return(rawCounts)
}
